In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
BASE_FOLDER = "compass 폴더의 위치를 지정해주세요"
target_data_path = os.path.join(BASE_FOLDER, "data/onehot_garments_pred.csv")

In [3]:
result_df = pd.read_csv(target_data_path)
result_df.head()

,RawText,LabelList,PredLogits
0,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,[1 0 1 0 0],[ 2.2611153 -2.0802803 2.0205135 -2.4404478 -...
1,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,[0 0 0 1 0],[-1.7596719 -0.87059915 -0.91030425 -1.624762...
2,그냥 착한가격이라 그냥 입기로 했지만 타사보다 사이즈가 좀크게 나왔어요,[0 1 1 0 0],[ 3.1217144 -3.3057523 -2.920321 0.377035...
3,딸다리가 길아 기모바지를 사주어도 다~ 내가 입게되어 속상했는데 기장도 길고 밍크...,[0 0 0 0 1],[-2.8631616 4.830046 -1.258243 -1.2631577 -...
4,직립금으로 좀 할인해서 구매했는데요. 블랙이 있으면 좋았겠지만 없어서 다크레드로 ...,[0 0 0 1 0],[ 4.275024 -2.9984868 -1.2868881 -2.426854...


In [4]:
def rollback_strfied_numpy(x):
    result = []
    for v in x[1:-1].strip().split(" "):
        if not v:
            continue
        result.append(float(v))
    return np.array(result)

result_df["LabelList"] = result_df["LabelList"].apply(rollback_strfied_numpy)
result_df["PredLogits"] = result_df["PredLogits"].apply(rollback_strfied_numpy)

In [5]:
result_df.head()

,RawText,LabelList,PredLogits
0,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,"[1.0, 0.0, 1.0, 0.0, 0.0]","[2.2611153, -2.0802803, 2.0205135, -2.4404478,..."
1,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,"[0.0, 0.0, 0.0, 1.0, 0.0]","[-1.7596719, -0.87059915, -0.91030425, -1.6247..."
2,그냥 착한가격이라 그냥 입기로 했지만 타사보다 사이즈가 좀크게 나왔어요,"[0.0, 1.0, 1.0, 0.0, 0.0]","[3.1217144, -3.3057523, -2.920321, 0.37703547,..."
3,딸다리가 길아 기모바지를 사주어도 다~ 내가 입게되어 속상했는데 기장도 길고 밍크...,"[0.0, 0.0, 0.0, 0.0, 1.0]","[-2.8631616, 4.830046, -1.258243, -1.2631577, ..."
4,직립금으로 좀 할인해서 구매했는데요. 블랙이 있으면 좋았겠지만 없어서 다크레드로 ...,"[0.0, 0.0, 0.0, 1.0, 0.0]","[4.275024, -2.9984868, -1.2868881, -2.4268546,..."


In [6]:
def logits_to_onehot_aspects(logits, threshold):
    sig_probabilities = tf.keras.activations.sigmoid(logits).numpy()
    return list(np.where(sig_probabilities > threshold, 1, 0))

aspect_bools = logits_to_onehot_aspects(result_df["PredLogits"].to_list(), 0.5)
pred_series = pd.Series(aspect_bools)
pred_series.head()

0    [1, 0, 1, 0, 0]
1    [0, 0, 0, 0, 1]
2    [1, 0, 0, 1, 0]
3    [0, 1, 0, 0, 0]
4    [1, 0, 0, 0, 1]
dtype: object

In [7]:
result_df["PredList"] = pred_series
result_df.head()

,RawText,LabelList,PredLogits,PredList
0,신발 편하고 예뻐요ᆢ 가성비 좋고 아무리 펀칭 되어 있어도 지금 신기엔 조...,"[1.0, 0.0, 1.0, 0.0, 0.0]","[2.2611153, -2.0802803, 2.0205135, -2.4404478,...","[1, 0, 1, 0, 0]"
1,착용감이 너무 편하고 좋아요 단추가 하나 떨어져서 오긴 했는데 그냥 고쳐서 입기...,"[0.0, 0.0, 0.0, 1.0, 0.0]","[-1.7596719, -0.87059915, -0.91030425, -1.6247...","[0, 0, 0, 0, 1]"
2,그냥 착한가격이라 그냥 입기로 했지만 타사보다 사이즈가 좀크게 나왔어요,"[0.0, 1.0, 1.0, 0.0, 0.0]","[3.1217144, -3.3057523, -2.920321, 0.37703547,...","[1, 0, 0, 1, 0]"
3,딸다리가 길아 기모바지를 사주어도 다~ 내가 입게되어 속상했는데 기장도 길고 밍크...,"[0.0, 0.0, 0.0, 0.0, 1.0]","[-2.8631616, 4.830046, -1.258243, -1.2631577, ...","[0, 1, 0, 0, 0]"
4,직립금으로 좀 할인해서 구매했는데요. 블랙이 있으면 좋았겠지만 없어서 다크레드로 ...,"[0.0, 0.0, 0.0, 1.0, 0.0]","[4.275024, -2.9984868, -1.2868881, -2.4268546,...","[1, 0, 0, 0, 1]"


In [8]:
test_series = result_df.apply(lambda x: x["LabelList"] == x["PredList"], axis=1)
test_series.head()

0        [True, True, True, True, True]
1      [True, True, True, False, False]
2    [False, False, False, False, True]
3      [True, False, True, True, False]
4     [False, True, True, False, False]
dtype: object

In [9]:
# 완전일치
def check_full_accord(x):
    for each in x:
        if not each:
            return 0
    return 1

test_series.apply(check_full_accord).mean()

0.041086663673102826

In [10]:
# 부분일치
def check_partial_accord(x):
    result = 0
    for each in x:
        if each:
            result += 1
    return result / 5

test_series.apply(check_partial_accord).mean()

0.5035866636731028